In [1]:
! git clone https://github.com/seshuad/IMagenet
! ls 'IMagenet/tiny-imagenet-200/'

fatal: destination path 'IMagenet' already exists and is not an empty directory.
test  train  val  wnids.txt  words.txt


In [2]:
from keras.models import load_model

model = load_model('exp1_best_model.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization_4 (Bat  (None, 32, 32, 64)        256       
 chNormalization)                                                
                                                                 
 conv2d_5 (Conv2D)           (None, 32, 32, 64)        36928     
                                                                 
 batch_normalization_5 (Bat  (None, 32, 32, 64)        256       
 chNormalization)                                                
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                      

In [3]:
import time
import imageio as nd
import numpy as np

path = 'IMagenet/tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open( path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict

def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open( path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [nd.imread( path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB') for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open( path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(nd.imread( path + 'val/images/{}'.format(img_name) ,mode='RGB'))
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    print('finished loading data, in {} seconds'.format(time.time() - t))
    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

train_data, train_labels, test_data, test_labels = get_data(get_id_dictionary())

print( "train data shape: ",  train_data.shape )
print( "train label shape: ", train_labels.shape )
print( "test data shape: ",   test_data.shape )
print( "test_labels.shape: ", test_labels.shape )

starting loading data


<ipython-input-3-37e7063e2103>:30: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  train_data += [nd.imread( path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB') for i in range(500)]
<ipython-input-3-37e7063e2103>:37: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  test_data.append(nd.imread( path + 'val/images/{}'.format(img_name) ,mode='RGB'))


finished loading data, in 47.607059955596924 seconds
train data shape:  (100000, 64, 64, 3)
train label shape:  (100000, 200)
test data shape:  (10000, 64, 64, 3)
test_labels.shape:  (10000, 200)


In [4]:
# Import necessary libraries
import tensorflow as tf
from keras.models import load_model
from keras.layers import Input, Dense, Flatten, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import LearningRateScheduler



# Assume new input size for Tiny ImageNet
new_input = Input(shape=(64, 64, 3))

# Create a new input layer that matches the size of Tiny ImageNet images
input_shape = (64, 64, 3)
new_input = Input(shape=input_shape)

model.trainable = True
# Pass new_input through the layers of the old model, except for the output layers
x = new_input
for layer in model.layers[:-3]:  # skip the last three layers (Flatten, Dense, Dense)
    x = layer(x)

# Calculate the correct output dimensions after the last pooling layer
shape_before_flatten = x.shape[1]
print("Shape before flattening:", shape_before_flatten)


# Flatten and adjust dense layers
# x = Flatten()(x)
x = Dense(1024, activation='relu', input_shape=(shape_before_flatten,), kernel_regularizer=l2(0.01))(x)  # Adjusted dense layer
x = Dropout(0.5)(x)
x = Dense(200, activation='softmax')(x)  # New output layer for 200 classes

# Create new model
new_model = Model(inputs=new_input, outputs=x)
new_model.summary()


# Learning rate scheduler function
def scheduler(epoch, lr):
    if epoch < 20:
        return lr
    else:
        return lr * np.exp(-0.1)

# Compile the new model
new_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])



Shape before flattening: 32768
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d_4 (Conv2D)           multiple                  1792      
                                                                 
 batch_normalization_4 (Bat  multiple                  256       
 chNormalization)                                                
                                                                 
 conv2d_5 (Conv2D)           multiple                  36928     
                                                                 
 batch_normalization_5 (Bat  multiple                  256       
 chNormalization)                                                
                                                                 
 max_pooling2d_2 (MaxPoolin  m

In [5]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')

# Ensure the data is in the correct shape and datatype
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')

# Normalize the data
train_data /= 255.0
test_data /= 255.0

# Verify shapes
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

# Create image data generators for data augmentation (optional but recommended)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

# Assuming the model is already compiled from your previous setup
# Define batch size and number of epochs
batch_size = 64
epochs = 50
# Callback for learning rate adjustment
lr_scheduler = LearningRateScheduler(scheduler)

# Train the model
history = new_model.fit(
    train_datagen.flow(train_data, train_labels, batch_size=batch_size),
    validation_data=test_datagen.flow(test_data, test_labels),
    epochs=epochs,
    steps_per_epoch=len(train_data) // batch_size,
    validation_steps=len(test_data) // batch_size,
    callbacks=[early_stopping, model_checkpoint, lr_scheduler]
)

# Optionally, save the trained model
new_model.save('trained_tiny_imagenet_model.h5')

# Evaluate the model
print(new_model.evaluate(test_datagen.flow(test_data, test_labels)))


Train data shape: (100000, 64, 64, 3)
Test data shape: (10000, 64, 64, 3)
Epoch 1/50
1562/1562 [==============================] - ETA: 0s - loss: 6.2265 - accuracy: 0.0368

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1562/1562 [==============================] - 127s 77ms/step - loss: 6.2265 - accuracy: 0.0368 - val_loss: 26.0162 - val_accuracy: 0.0042 - lr: 0.0010
Epoch 2/50
1562/1562 [==============================] - 121s 78ms/step - loss: 5.9041 - accuracy: 0.0670 - val_loss: 6.7537 - val_accuracy: 0.0373 - lr: 0.0010
Epoch 3/50
1562/1562 [==============================] - 121s 78ms/step - loss: 5.6787 - accuracy: 0.0865 - val_loss: 5.6714 - val_accuracy: 0.0871 - lr: 0.0010
Epoch 4/50
1562/1562 [==============================] - 119s 76ms/step - loss: 5.5708 - accuracy: 0.0956 - val_loss: 13.6067 - val_accuracy: 0.0062 - lr: 0.0010
Epoch 5/50
1562/1562 [==============================] - 120s 77ms/step - loss: 5.4893 - accuracy: 0.1012 - val_loss: 11.2006 - val_accuracy: 0.0122 - lr: 0.0010
Epoch 6/50
1562/1562 [==============================] - 120s 77ms/step - loss: 5.4400 - accuracy: 0.1050 - val_loss: 8.4337 - val_accuracy: 0.0076 - lr: 0.0010
Epoch 7/50
1562/1562 [==========================

In [6]:
import pandas as pd

pd.DataFrame(history.history).to_csv('trained_tiny_imagenet_history.csv')